# Getting started
This notebook demonstrates the basic functionality of the HPS API and shows how to start a training session and run evaluations. 

In [ ]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

In [ ]:
import pandas as pd
import numpy as np
#from hps_client_sdk.apiproxy import ApiProxy
from matplotlib import pyplot as plt
from hps_api_client.apiproxy import ApiProxy

## Create client
 - The ApiProxy client facilitates access to the api functions and provides classes that wrap the json objects which are exchanged with the server
 - The proxy is preconfigured to point at the api

In [ ]:
client = ApiProxy(uri = "http://leviathan:5400/api/v1/")
#help(ApiProxy)

## List available projects
 - Projects are a means to organize trainig sessions
 - All projects created by any user will be listed

In [ ]:
projects = client.get_projects()
projects

## List hydrosystems
Hydro systems are pre-configured for our customers and cannot be added through the API.
 

In [ ]:
hydro_systems = client.get_hydro_systems()
hydro_system = hydro_systems[0]
hydro_systems

## Create a new project
For this tutorial we will create a new project for the hydro system Flørli

In [ ]:
project_name = "Comp TF vs SB 2*364 days"
selected_hydro_system = hydro_systems[0]
#new_project = client.create_project(project_name, selected_hydro_system)
new_project = projects[-1]

## Get forecasts
We are going to pick a forecast to train our agent on. First we list available forecasts.

In [ ]:
forecasts = client.get_forecasts(selected_hydro_system)
forecasts

We will use the most recent forecast 

In [ ]:
selected_forecast = forecasts[-1]
selected_forecast

## Get reservoirs for hydro system

In [ ]:
reservoirs = client.get_reservoirs(selected_hydro_system)
reservoirs

## Start training
When starting a training run, a settings object has to be provided. A template for these settings can be retrieved from the API.

In [ ]:
settings = client.get_settings_template(new_project)
print(settings.to_json())

Before starting the run, we will modify some of the settings

In [ ]:
from core.noise import Noise
from hps.rl.environment.end_value_type import EndStateIncentive
from server.setting_combiner import StepResolution

# It is impotant to give the run a description, so we can identfy it later
TRAIN_STEPS = 10000000

settings.stepsInEpisode = 364*4
settings.trainEpisodes = TRAIN_STEPS//settings.stepsInEpisode
settings.evaluationInterval = 30
settings.rewardScaleFactor = 1
settings.noise = Noise.White
settings.endStateIncentive = EndStateIncentive.ProvidedEndEnergyPrice
settings.endEnergyPrice = 30
settings.stepResolution = StepResolution.Day
settings.evaluationEpisodes = 30

settings.comment = "SAC SB SDE+OUAN+LRDEC 1 price + lin + vol + flow"
settings.algo = "SAC"
# We are setting all initial reservoir volums to half capasity
settings.startVolumes = {}
for r in reservoirs:
    settings.startVolumes[r.name] = r.maxVolume / 2
    
    
print(settings.to_json())

We can now start the trainig session in the context of our new project and using the forcast and settings as specified above

In [ ]:
run = client.run(new_project, selected_forecast, settings)
run

In [ ]:
client.terminate_run(runs[-1])

As we can see above, initially the run has neither start- nor endTime specified. As soon as the backend starts the training process, the startTime will be set. When the training process has completed the endTime will also be specified.
We can now call get_runs for our project to the status of our run.

In [ ]:
runs = client.get_runs(new_project)
runs

In [ ]:
# Runs 5 and 6 are L2 5e-2, not 5e-3
runs = client.get_runs(new_project)
from datetime import datetime as dt
from dateutil.parser import parse
import pytz

for run in runs:
    if run.endTime is None:
        end_time = dt.utcnow().replace(tzinfo=pytz.utc)
    else:
        end_time = parse(run.endTime)
    start_time = parse(run.startTime)
    print(end_time - start_time, " ", run.name)

During training it can be useful to monitor the progress. The show_progress method will display the maximum reward achieved  over time, and can be a good way to identify convergence.

In [ ]:
def plot_and_analyze(run, tag = "Test Return"):
    data = client.api_client.get_run_details(run.uid, tag=tag, plot_only=False)
    plt.grid()
    means = []
    lasts = []
    for i, d in enumerate(data):       
        means.append(np.mean(d["values"]))
        lasts.append(d["values"][-1])
    print(np.mean(means), np.mean(lasts), np.min(lasts), np.max(lasts))
    
def plot_train_return(run):
    plot_and_analyze(run, "Train return")
    
def plot_loss(run):
    plot_and_analyze(run, "Loss")

In [ ]:
def moving_average(a, n=20) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

data = client.api_client.get_run_details(runs[-1].uid, tag="Test Return", plot_only=False)
#data = client.api_client.get_run_details(runs[-1].uid, tag="Critic loss", plot_only=False)
#data = client.api_client.get_run_details(runs[-2].uid, tag="Critic loss", plot_only=False)
for d in data:
    xs = d["steps"]
    ys = d["values"]
    N = 100
#    print(ys)
#    ys = np.convolve(ys, np.ones(N)/N, mode='valid')
#    print(ys)
    plt.plot( ys)

In [ ]:
for d in data:
    xs = d["steps"]
    ys = moving_average(d["values"], n=5)
    plt.plot(ys)

In [ ]:
plt.subplots(1)
plot_and_analyze(runs[-1])
plt.subplots(1)
plot_and_analyze(runs[-1], tag="Best return")

In [ ]:
#plot_loss(runs[-2]) 
plot_and_analyze(runs[-1], tag ="Critic loss") #plot_train_return(runs[-1]

In [ ]:
#plot_loss(runs[-2]) 
plot_and_analyze(runs[-1], tag ="Actor loss") #plot_train_return(runs[-1])

In [ ]:
def fetch_data(uid, rsf=1.0):
    cli = client.api_client
    details_uri = cli.uri + "projectruns/" + uid + "/rundetails"
    details, _ = cli.fetch(details_uri)

    test_return_datas = [d for d in details["status"] if d["name"] == "Test Return"]
    
    result = {}
    for dd in test_return_datas:
        for step, val in zip(dd["steps"], dd["values"]):
            if not step in result:
                result[step] = []
            result[step].append(val/rsf)
    
    return result

In [ ]:
def get_plot_data(uid, rsf):
    data = fetch_data(uid, rsf)
    steps = [int(v) for v in data.keys()]

    xs = []
    ys = []
    max_val = 0
    for s in steps:
        val = max(data[s])
        if val > max_val:
            max_val = val
            xs.append(s)
            ys.append(val)
    return xs, ys

In [ ]:
def get_stat_data(uid, rsf, max_step=500000):
    data = fetch_data(uid, rsf)
    steps = [int(v) for v in data.keys()]

    xs = []
    means = []
    mins = []
    maxs = []
    dev = []
    
    max_val = 0
    for s in steps:
        if s > max_step: break
        xs.append(s)
        means.append(np.mean(data[s]))
        dev.append(np.std(data[s]))
        maxs.append(np.max(data[s]))
        mins.append(np.min(data[s]))
    return np.array(xs), np.array(means), np.array(mins), np.array(maxs), np.array(dev)

In [ ]:
import matplotlib.colors as mcolors
runs = client.get_runs(new_project)
colors = list(mcolors.TABLEAU_COLORS)
markers = ["+", "x", "v", "o", "P", "H", "h", "o", ".", "+", "x", "v", "o", "P", "H", "h", "o", "."]
best = []
last = []

fix, ax = plt.subplots(figsize=(20,10))
ind = 0

sel_runs = []
for i, r in enumerate(runs):
    if not i in [50] : sel_runs.append(r)

STDDEV = False

for i, prun in enumerate(sel_runs):    
    xs, ys, mins, maxs, dev = get_stat_data(prun.uid, prun.settings.rewardScaleFactor, max_step = 1600000)
                    
    if len(ys) > 0:
        plt.plot(xs, ys, "-" + markers[ind], label=prun.settings.comment, color = colors[ind])
        if STDDEV:            
            plt.fill_between(xs, ys - dev, ys + dev, color=colors[ind], alpha=0.2)
        else:
            plt.fill_between(xs, mins, maxs, color=colors[ind], alpha=0.2)
        best.append(np.max(maxs))
        last.append((prun.name, xs[-1], ys[-1]))
    ind += 1
    
plt.legend(loc='lower left')
plt.grid(True)
#ax.set_ylim([13.5,15.5])
print(best, np.max(best))
print(last)

In [ ]:
x = np.array([0,1,2,3,4,5,6,7,8,9])
for a in range(520):
#    plt.plot(action_noise.call())
    x = x +21
    
    
noise = 0.1*np.random.normal(0.0,0.1,5)
noise

plt.plot(0.1 * (9 * np.linspace(1.0,0.0,20) + 1) * 1e-3)

In [ ]:
r = runs[-1]
client.plot_solution(r)
r.name

In [ ]:
r = runs[-2]
client.plot_solution(r)
r.name

At any time during or after training we can retrieve the best solution produced by the agent.

In [ ]:
sol1 = client.get_solution(runs[-2])

In [ ]:
price = sol1["Energy_Price"]
power = sol1["Sum_MWh"]
vol = sol1["Flørlivatn"]

price_std = price.std(axis = 1).values
price_mean = price.mean(axis = 1).values

prod_std = power.std(axis = 1).values
prod_mean = power.mean(axis = 1).values

vol_std = vol.std(axis = 1).values
vol_mean = vol.mean(axis = 1).values


x = np.arange(0, len(price))

_,ax = plt.subplots(3, 1,figsize=(20,10))


ax[0].plot(x, price_mean, "-" + markers[0], label="Price", color = colors[0])
ax[0].fill_between(x, price_mean - price_std, price_mean + price_std, color=colors[0], alpha=0.2)

ax[1].plot(x, vol_mean, "-" + markers[0], label="Price", color = colors[0])
ax[1].fill_between(x, vol_mean - vol_std, vol_mean + vol_std, color=colors[0], alpha=0.2)

ax[2].plot(x, prod_mean, "-" + markers[0], label="Price", color = colors[0])
ax[2].fill_between(x, prod_mean - prod_std, prod_mean + prod_std, color=colors[0], alpha=0.2)


In [ ]:
sol2 = client.get_solution(runs[-3])

In [ ]:
df = pd.DataFrame(sol2["Sum_money"].sum())
df["1"] = sol1["Sum_money"].sum()

df.columns = ["with", "without"]
df.plot.bar(figsize=(20,20))
#sol2["Sum_MWh"].sum().plot.bar()
#sol1["Sum_MWh"].sum().plot.bar()

#- sol1["Sum_MWh"].sum()).plot.bar(figsize=(20,20))


In [ ]:
price = sol2["Energy_Price"]
power = sol2["Sum_MWh"]
vol = sol2["Flørlivatn"]

price_std = price.std(axis = 1).values
price_mean = price.mean(axis = 1).values

prod_std = power.std(axis = 1).values
prod_mean = power.mean(axis = 1).values

vol_std = vol.std(axis = 1).values
vol_mean = vol.mean(axis = 1).values

x = np.arange(0, len(prod_std))

_,ax = plt.subplots(3, 1,figsize=(20,10))


ax[0].plot(x, price_mean, "-" + markers[0], label="Price", color = colors[0])
ax[0].fill_between(x, price_mean - price_std, price_mean + price_std, color=colors[0], alpha=0.2)

ax[1].plot(x, vol_mean, "-" + markers[0], label="Price", color = colors[0])
ax[1].fill_between(x, vol_mean - vol_std, vol_mean + vol_std, color=colors[0], alpha=0.2)

ax[2].plot(x, prod_mean, "-" + markers[0], label="Price", color = colors[0])
ax[2].fill_between(x, prod_mean - prod_std, prod_mean + prod_std, color=colors[0], alpha=0.2)


In [ ]:
price = sol2["Energy_Price"]
power = sol2["Sum_MWh"]
vol = sol2["Flørlivatn"]

price_std = price.std(axis = 1).values
price_mean = price.mean(axis = 1).values

prod_std = power.std(axis = 1).values
prod_mean = power.mean(axis = 1).values

vol_std = vol.std(axis = 1).values
vol_mean = vol.mean(axis = 1).values

x = np.arange(0, len(prod_std))

_,ax = plt.subplots(5, 1,figsize=(20,10))


ax[0].plot(x, price_mean, "-" + markers[0], label="Price", color = colors[0])
ax[0].fill_between(x, price_mean - price_std, price_mean + price_std, color=colors[0], alpha=0.2)

ax[1].plot(x, vol_mean, "-" + markers[0], label="Price", color = colors[1])
ax[1].fill_between(x, vol_mean - vol_std, vol_mean + vol_std, color=colors[1], alpha=0.2)

ax[2].plot(x, prod_mean, "-" + markers[0], label="Price", color = colors[1])
ax[2].fill_between(x, prod_mean - prod_std, prod_mean + prod_std, color=colors[1], alpha=0.2)

power = sol1["Sum_MWh"]
vol = sol1["Flørlivatn"]
prod_std = power.std(axis = 1).values
prod_mean = power.mean(axis = 1).values
vol_std = vol.std(axis = 1).values
vol_mean = vol.mean(axis = 1).values

ax[3].plot(x, vol_mean, "-" + markers[0], label="Price", color = colors[2])
ax[3].fill_between(x, vol_mean - vol_std, vol_mean + vol_std, color=colors[2], alpha=0.2)

ax[4].plot(x, prod_mean, "-" + markers[0], label="Price", color = colors[2])
ax[4].fill_between(x, prod_mean - prod_std, prod_mean + prod_std, color=colors[2], alpha=0.2)


In [ ]:
def get_money(run, reservoirs, end_price):
    solution = client.api_client.best_solution(run.uid)
    solution = ApiReportData(solution)
    data = client.build_data_frame(solution)
    money = data["Sum_money"].sum().sum()
    
    r_sum = 0
    for r in reservoirs:
        sub = data[r.name]
        
        for col in sub.columns:
            r_sum += sub[col].values[-1]
    
    rest = r_sum * end_price * 10**3 * 1.8491997820234045
    return {"production" : money, "rest" :  rest, "sum" : money + rest}
    

In [ ]:
labels = []
prod = []
rest = []


for i, run in enumerate(runs.values):
    if not i in [2,3]: continue 
    res = get_money(run, reservoirs, settings.endEnergyPrice)
    print(run.name, res["sum"])
    labels.append(run.name)
    prod.append(res["production"])
    rest.append(res["rest"])
    
width = 0.25       # the width of the bars: can also be len(x) sequence

fig, ax = plt.subplots(figsize=(20,10))

print(len(prod),len(rest))

ax.bar(labels, prod, width, label='production')
ax.bar(labels, rest, width, bottom=prod, label='Rest')

ax.set_ylabel('Reward')
ax.set_ylim([1261000000,1267000000])
ax.legend()

plt.grid(True)
plt.show()

In [ ]:
1264370646.0746615 - rest[0]


In [ ]:
data["Flørlivatn"].columns

A training session can be terminated before it has run to completion.

In [ ]:
client.terminate_run(runs[-1])

In [ ]:
print(settings.to_json())

## Evaluation
During training we normally evaluate only a subset of the forecast scenarios to determine convergence of the solution. After or during training we can perform more detailed evaluations.

In [ ]:
evaluation = client.evaluate(runs[-1], selected_forecast, settings)
evaluation

As with a trainig session, the evaluation is completed when the endTime is set.

In [ ]:
client.get_evaluations(new_project)

We can plot the results of the evaluation

In [ ]:
client.plot_evaluation(evaluation)

And for further processing, we can retrieve the result data as a pandas DataFrame

In [ ]:
df = client.get_evaluation(evaluation)
df

In [ ]:
df_mean = df

In [ ]:
df_mean["Flørlivatn"].plot(figsize=(20,10))
#df_min_max[("Flørlivatn", "1990")].plot(figsize=(20,10))

In [ ]:
df_mean["Flørlivatn"].plot(figsize=(20,10))


In [ ]:
df_min_max["Flørlivatn"].plot(figsize=(20,10))

In [ ]:
def comp_enhs(vals, fac, mid_norm = 0.5):
    minv = np.min(vals)
    maxv = np.max(vals)
    mid = minv + (maxv-minv)/2
        
    norm_vals = (vals - minv) / (maxv - minv)
    pow_vals = pow(norm_vals, fac)

    norm_vals = pow_vals * (maxv - minv) + minv
    
    if mid_norm is not None:
        norm_vals = norm_vals - mid * mid_norm
        
    return norm_vals

vals = [10,12,15,18, 22, 35, 40, 42, 43, 44, 33, 12, 23, 23, 11]
vals = 5* np.sin(vals) + vals

plt.plot(vals)
plt.plot(comp_enhs(vals, 2, None))
plt.plot(comp_enhs(vals, 2, 0.5))


In [ ]:
print(vals)
print(comp_enhs(vals, 2, None))